# COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases by country. Additionally, a detailed view is provided for the US(by state), Europe

- comments: true
- author: Pratap Vardhan
- categories: [overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [2]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [3]:
#hide

# FETCH
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [4]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

,Country/Region,Cases,Deaths,PCases,PDeaths,Cases (+),Deaths (+),Fatality Rate,Continent
0,US,17206647,310699,16143219,297904,1063428,12795,1.8,North America
1,India,9979447,144789,9857029,143019,122418,1770,1.5,Asia
2,Brazil,7110434,184827,6880127,181123,230307,3704,2.6,South America
3,Russia,2736727,48568,2602048,45923,134679,2645,1.8,Europe
4,France,2483524,59733,2407079,57671,76445,2062,2.4,Europe
5,Turkey,1955680,17364,1809809,16199,145871,1165,0.9,Asia
6,United Kingdom,1954268,66150,1835949,64123,118319,2027,3.4,Europe
7,Italy,1906377,67220,1825775,64036,80602,3184,3.5,Europe
8,Spain,1785421,48777,1730575,47624,54846,1153,2.7,Europe
9,Argentina,1524372,41534,1494602,40668,29770,866,2.7,South America


In [6]:
#hide
#delete problematic countries from table
df_table = df_table[~df_table['Country/Region'].isin(['Cape Verde', 'Cruise Ship', 'Kosovo'])]

In [7]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'updated': Timestamp('2020-12-17 00:00:00'),
 'since': Timestamp('2020-12-12 00:00:00'),
 'Cases': 74904722,
 'Deaths': 1660902,
 'Cases (+)': 3163211,
 'Deaths (+)': 56985,
 'China Cases': 94734,
 'China Deaths': 4759,
 'China Cases (+)': 517,
 'China Deaths (+)': 10,
 'US Cases': 17206647,
 'US Deaths': 310699,
 'US Cases (+)': 1063428,
 'US Deaths (+)': 12795,
 'EU Cases': 20722952,
 'EU Deaths': 478478,
 'EU Cases (+)': 1046433,
 'EU Deaths (+)': 24962}

In [8]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,200,135,202,63,113,298,191,234,209,217
Albania,0,0,0,0,0,0,0,0,0,0,...,753,752,873,802,879,788,661,809,637,787
Algeria,0,0,0,0,0,0,0,0,0,0,...,591,598,565,542,517,464,495,468,442,426
Andorra,0,0,0,0,0,0,0,0,0,0,...,43,35,28,46,52,50,44,0,64,20
Angola,0,0,0,0,0,0,0,0,0,0,...,81,75,121,136,100,27,89,85,45,77


In [9]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/)